In [1]:
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [3]:
#passing column names for each CSV as the column name is not given in thr file

#reading users files

u_cols=['user_id', 'age', 'sex','occupation', 'zip_code']

userd=pd.read_csv('user.csv', sep='|' , names=u_cols, encoding='latin-1')

In [5]:
print(userd.shape)
userd.head()

(943, 5)


,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [6]:
#reading ratings file

r_cols=['user_id','movie_id','rating','unix_timestamp' ]

ratings=pd.read_csv('ratings.csv', sep='\t', names=r_cols, encoding='latin-1')

print(ratings.shape)
ratings.head()

(100000, 4)


,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [8]:
#reading items file

i_cols=['movie_id','movie_title','release_date','video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure','Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy','Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

items=pd.read_csv('items.csv', sep='|', names=i_cols , encoding='latin-1')

print(items.shape)
items.head()

(1682, 24)


,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [9]:
#dataset has already been divided into train and test by GroupLens where the test data has 10 ratings for each user

r_cols=['user_id','movie_id','rating','unix_timestamp' ]
ratings_train=pd.read_csv('userBase.csv', sep='\t', names=r_cols, encoding='latin-1')
ratings_test=pd.read_csv('userTest.csv', sep='\t', names=r_cols, encoding='latin-1')

ratings_train.shape , ratings_test.shape

((90570, 4), (9430, 4))

In [10]:
#calculating the no. of unique users and items

n_users= ratings.user_id.unique().shape[0]
n_items= ratings.movie_id.unique().shape[0]



In [12]:
#creating user-item matrix ehich can be used to calculate similarity between users and items

data_matrix = np.zeros((n_users,n_items))
for line in ratings.itertuples():
    data_matrix[line[1]-1,line[2]-1]=line[3]

In [14]:
#using pairwise_distance function from sklearn to calcilate cosine similarity

from sklearn.metrics.pairwise import pairwise_distances

user_similarity = pairwise_distances(data_matrix, metric ='cosine')
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')

In [17]:
#defining predict function based on above similarity

def predict(ratings, similarity, type='user'):
    if type=='user':
        mean_user_rating=ratings.mean(axis=1)
        ratings_diff= (ratings-mean_user_rating[:,np.newaxis])
        pred=mean_user_rating[:, np.newaxis]+similarity.dot(ratings_diff)/np.array([np.abs(similarity).sum(axis=1)]).T
    elif type== 'item':
        pred=ratings.dot(similarity)/np.array([np.abs(similarity).sum(axis=1)])
    return pred    

In [19]:
user_prediction = predict(data_matrix, user_similarity, type='user')
item_prediction = predict(data_matrix, item_similarity, type='item')

In [20]:
print(user_prediction)

[[ 2.06532606  0.73430275  0.62992381 ...  0.39359041  0.39304874
   0.3927712 ]
 [ 1.76308836  0.38404019  0.19617889 ... -0.08837789 -0.0869183
  -0.08671183]
 [ 1.79590398  0.32904733  0.15882885 ... -0.13699223 -0.13496852
  -0.13476488]
 ...
 [ 1.59151513  0.27526889  0.10219534 ... -0.16735162 -0.16657451
  -0.16641377]
 [ 1.81036267  0.40479877  0.27545013 ... -0.00907358 -0.00846587
  -0.00804858]
 [ 1.8384313   0.47964837  0.38496292 ...  0.14686675  0.14629808
   0.14641455]]
